In [1]:
#Requires Python 3.8+ for inverse mod using pow()

# Imports

In [2]:
import random
import secrets
from decimal import *

# Functions

Returns the GCD of 2 numbers

In [3]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

Rabin-Miller Primality Test<br>
Checks if a number is prime, p is the number to be tested, s is the number of rounds<br>
Number of rounds should ensure that the probability of p not being prime is < 2^(-80) 

In [4]:
def RMPT(p, s):
    q = p - 1
    k = 0
    while q % 2 != 1:
        q //= 2
        k += 1
        
    for _ in range(s):
        a = random.randrange(3, p - 2)      
        while gcd(a, p) != 1:
            a = random.randrange(3, p - 2)
        
        z = pow(a, q, p)
        
        if z == 1 or z == p - 1:
            continue #Not a witness, pick another a
        
        for i in range(1, k):
            z = pow(a, pow(2, i)*q, p)
            if z == p - 1:
                break #Not a witness, pick another a
        else:
            return False  #a^((2^K)*Q)modP != -1 for all values of K, it is a witness, therefore its definitely not prime
        
    return True #After S iterations still not a composite, chance to be a prime

Converts a string to ascii

In [5]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

Converts ascii back to string

In [6]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

Generates the decryption key using public key e and 2 prime numbers p and q

In [7]:
def RSA_generate_D(e, p, q):
    N = p * q
    phi_N = (p - 1) * (q - 1)
    assert gcd(e, phi_N) == 1, "GCD of e and phi N must be 1"
    d = pow(e, -1, phi_N)
    return d

Encrypts a number with public key e

In [8]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

Encrypts a number using private key d

In [9]:
def RSA_decrypt(encrypted_message, d, N):
    ascii_message = pow(encrypted_message, d, N)
    return ascii_message

Continuously generates random bit_length numbers<br>
If number is even, adds one to make it odd<br>
Tries to factorize the number with small primes to determine if composite<br>
Uses Rabin-Miller Primality Test if it is still possible to be a prime number

In [10]:
def Generate_Prime(bit_length, RMPT_rounds):   
    while(True):
        num = secrets.randbits(bit_length)
        
        #If number is even, make it odd
        if num%2 == 0:
            num += 1
        #Try factorize using small primes
        elif num%3 == 0:
            continue
        elif num%5 == 0:
            continue
        elif num%7 == 0:
            continue
        elif num%11 == 0:
            continue
        #Try the expensive Rabin-Miller Primality Test
        elif RMPT(num, RMPT_rounds):
            print(num)
            return num

In [12]:
#Workaround for getting the cube root of a huge integer

def cube_root(encrypted_message):
    minprec = 27

    encrypted_message_len = len(str(encrypted_message))

    if encrypted_message_len > minprec: 
        getcontext().prec = encrypted_message_len
    else:
        getcontext().prec = minprec

    encrypted_message = Decimal(encrypted_message)
    power = Decimal(1)/Decimal(3)

    #Gets cube root of encrypted message
    decrypted_message = encrypted_message**power
    decrypted_message = int(decrypted_message.quantize(Decimal('1.'), rounding=ROUND_UP))
    
    return decrypted_message

# RSA 1024

Let's try to encrypt a message using RSA 1024

Generate the primes p q

Use encryption exponent of 3

Get decryption exponent

In [15]:
N_test=False
e = 3

while (N_test==False):
    p = Generate_Prime(1024, 3)
    q = Generate_Prime(1024, 3)
    if (p%e==2 and q%e==2):
        N_test=True
        
d = RSA_generate_D(e, p, q)
N = p*q

112588216933712141833313659369581251391630663547102258784653079438460561505802995325408504468058806831639122071319708672182025364815269766213872880033386354133333383246516794596138550124122446934990547646603568497485101118370199993016844160244641141390544027025305543263615034868758613179218543543889325249819
84974291671954344442838089942265385184205825692749121866005599299341152321517076421862279342400068139382458911907533561410695137359694809596024008687398476228526476783947292853544301434676557523582030508698348061627424403938888390105104334528273634192955959752957371983072079958083884987762040393037735903881
65988679970268430850912068736224806199661825407235101773604210196317085353658554151647824718500114682221912826579056651411055210797178028729226193132469715304591432889638779857317481246441508852518450610599811586759924011070479349335082442081824750830889381153699064315938093417122873947962293898757719450943
890366881843681138014934696192407721567121727398882420308024367827516695

In [14]:
message = "Hello, this message is secure in RSA1024 but weak in RSA 2048 without padding with a small exponent of 3"

ascii_message = convert_to_ascii(message)
print(ascii_message)
ascii_message.bit_length()

72101108108111044032116104105115032109101115115097103101032105115032115101099117114101032105110032082083065049048050052032098117116032119101097107032105110032082083065032050048052056032119105116104111117116032112097100100105110103032119105116104032097032115109097108108032101120112111110101110116032111102032051


1033

In [16]:
encrypted_message = RSA_encrypt(ascii_message, e, N)

In [17]:
print(encrypted_message)

528398647669888288186267753454603680742288977310168927459929307904701676455173556588457727593511499770391924114250910337552762305307445237038765407705218713151645608916235392115708082517776391908682103441943926203178953965945864873253821785979382921476973125534837871909691098606108707720785255483357262153528021303838825202689323394855391222334052982154514309341501130964236689825620161536717130722570873443067253958062819893990169458845595378108359823811352711689104597106868555469975794805064671352750317248762861441238660136994467354597465940390125704167295243134810311637848329452412908487636863234639257844663


Python is unable to handle such large numbers for cube root

In [18]:
decrypted_message = encrypted_message ** (1/3)

OverflowError: int too large to convert to float

Use workaround function to get cube root

In [19]:
decrypted_message = cube_root(encrypted_message)

print(convert_to_string(decrypted_message))

TȁʓǙĶŁΑ̝ɯǋ̘Ɔ˕̲Ŷș˽(=#ũ΅ΐȔėϝǉÝIʸϊΙϡͥ̄ɯʵĦǀǓϏſĸȯĆĎŕ˅Ϗͽ˟͹ɝioʖʨǚŘτǛŝͽ;̀Ͱ


Message is still encrypted even after cube root

Lets properly decrypt the message now

In [20]:
decrypted_message = RSA_encrypt(encrypted_message, d, N)

In [21]:
print(convert_to_string(decrypted_message))

Hello, this message is secure in RSA1024 but weak in RSA 2048 without padding with a small exponent of 3


# RSA 2048

Let's try to encrypt a message using RSA 2048

Generate the primes p q

Use encryption exponent of 3 again

Get decryption exponent

In [22]:
N_test=False
e = 3

while (N_test==False):
    p = Generate_Prime(2048, 3)
    q = Generate_Prime(2048, 3)
    if (p%e==2 and q%e==2):
        N_test=True
        
d = RSA_generate_D(e, p, q)
N = p*q

19638157927142284852216492901946221415788364077789220895501387089023734458491250113825478798508113814176121520562383582904245350856310956531279624785550973472321051193011196915890479097299242156877220401728416914249210932913718255543361068356703292493766732085110704239532885661586082003054253308068220077804801835839356609541754607413914238109848113300640479217137841592410828822610145385941628102514413959364473251662124363752910904378861797469234750695312279509679338135466949157129605893009651487970308352279697560298433276280410387352855000199888062834180673102372316493066085704322432815831231516377948516062739
2923728985028249147832524032885661689775063891395896597588598724094442743303672421835898871852886726630404373152728144996133564822196491366444343805318228337672671857512548604325042819465139027683469885079842062704583243642794880837927456188853040983337956300740991093481702009510112956486102193343499399889692055159741417392571687129929835057758452646333583073584886061768734804280

In [23]:
message = "Hello, this message is secure in RSA1024 but weak in RSA 2048 without padding with a small exponent of 3"

ascii_message = convert_to_ascii(message)
print(ascii_message)
ascii_message.bit_length()

72101108108111044032116104105115032109101115115097103101032105115032115101099117114101032105110032082083065049048050052032098117116032119101097107032105110032082083065032050048052056032119105116104111117116032112097100100105110103032119105116104032097032115109097108108032101120112111110101110116032111102032051


1033

In [24]:
encrypted_message = RSA_encrypt(ascii_message, e, N)

In [25]:
print(encrypted_message)

374822642466453503471129234953054526212879179388354880533829447986843654790252314554033043646746764449913639182639657708742449037448950714078946330611043527252186205143428991207943778445949669261828678244029033364957051851646843200248207959737882920354740993629980412340688249208986011917884001719076791515449239894248936982703403870041158347994141996971536956348876306054595083780967728076706532035714204633008080071541961769864324054181130056979714597750740643162189705244532503479892077386565813438424295161109106397041214616802454831180487114975435436195881465431700791503705277369993863949588713707081687514834613488238651365075709738120939125370934830072994404453950653829807147232013328123932604129717807697923904573366449573757101712631775756908010422087048442049253902421391121963980754726364178279941244392279920641838047656797066648613408324642902777263968839370282113878069675238833717309324302239332461043662637827828651


In [26]:
print(convert_to_string(encrypted_message))

ň{Τɜˍ̧ʹΛΈȽŮǁȽ˵eˈɷ̇˴Ό
ƦW0ƺ1ýΆƥƇyσϔ˲˖Ŭ²ėέôƈėΘʁ͆/ʐ̝BʈɥƘńʂΆ̉ćψ͇ŲĚqͮEʣî́ˍĵńĮïŌǍ+ʖɽ̻̼ʋ


Since the number of bits of the plain text is much smaller than 2048 bits, even when the plain text raised to the power of 3 it is still smaller than 2048 bits which means the mod operation does nothing, and i can just use the cube root to find the original message

In [27]:
decrypted_message = cube_root(encrypted_message)

In [28]:
print(convert_to_string(decrypted_message))

Hello, this message is secure in RSA1024 but weak in RSA 2048 without padding with a small exponent of 3


# Conclusions

This exploit can be avoided by simply padding the original message to make it the same size as N or by choosing a larger exponent